### Index

> 1. __Environment setup__ - Import packages, set user defined inputs, implement the hub methods in python that fetches initiaitves and followers

> 2. __Fetch initiaitves (topic names)__

> 3. __Implement methods making necessary GovDelivery API calls__

> 4. __Main script that fetches new followers and adds them as subscribers if needed__

### 1. Environment setup

In [1]:
from arcgis.gis import GIS
from arcgis._impl.common._mixins import PropertyMap
import collections
import json
import requests
import xml.etree.ElementTree as ET
import os.path
import datetime

In [2]:
#Inputs for Enterprise and Community organizations of the Hub
e_org_url = "enterprise org url to your Hub"
e_username = "admin"
e_pasword = "password"

c_org_url = "community org url to your Hub"
c_username = "admin_community"
c_password = "password"

#Govdelivery API credentials
gd_username = 'username'
gd_password = 'password'

#Frequency (minutes) of executing - 24 hours
minutes = 1440

In [3]:
def _lazy_property(fn):
    '''Decorator that makes a property lazy-evaluated.
    '''
    # http://stevenloria.com/lazy-evaluated-properties-in-python/
    attr_name = '_lazy_' + fn.__name__

    @property
    def _lazy_property(self):
        if not hasattr(self, attr_name):
            setattr(self, attr_name, fn(self))
        return getattr(self, attr_name)
    return _lazy_property

class Hub(object):
    """
    Entry point into the Hub module. Lets you access an individual hub and its components.
       
    ================    ===============================================================
    **Argument**        **Description**
    ----------------    ---------------------------------------------------------------
    url                 Required string. If no URL is provided by user while connecting 
                        to the GIS, then the URL will be ArcGIS Online.
    ----------------    ---------------------------------------------------------------
    username            Optional string as entered while connecting to GIS. The login user name 
                        (case-sensitive).
    ----------------    ---------------------------------------------------------------
    password            Optional string as entered while connecting to GIS. If a username is 
                        provided, a password is expected.  This is case-sensitive. If the password 
                        is not provided, the user is prompted in the interactive dialog.
    ================    ===============================================================
    """
    
    def __init__(self, url, username=None, password=None):
        #self.gis = gis
        self._username = username
        self._password = password
        self.url = url
        self.gis = GIS(self.url, self._username, self._password)
        try:
            self._gis_id = self.gis.properties.id
        except AttributeError:
            self._gis_id = None
            
    @property
    def enterprise_org_id(self):
        """
        Returns the AGOL org id of the Enterprise Organization associated with this Hub.
        """
        try:
            self.gis.properties.portalProperties.hub
            try:
                return self.gis.properties.portalProperties.hub.settings.enterpriseOrg.orgId
            except AttributeError: 
                return  self._gis_id
        except:
            print("Hub does not exist or is inaccessible.")
            raise
                        
    @property
    def community_org_id(self):
        """
        Returns the AGOL org id of the Community Organization associated with this Hub.
        """
        try:
            self.gis.properties.portalProperties.hub
            try:
                return self.gis.properties.portalProperties.hub.settings.communityOrg.orgId
            except AttributeError:
                return  self._gis_id
        except:
            print("Hub does not exist or is inaccessible.")
            raise  
  
    @property
    def enterprise_org_url(self):
        """
        Returns the AGOL org url of the Enterprise Organization associated with this Hub.
        """
        try:
            self.gis.properties.portalProperties.hub
            try:
                self.gis.properties.portalProperties.hub.settings.enterpriseOrg
                try:
                    _url = self.gis.properties.publicSubscriptionInfo.companionOrganizations[0]['organizationUrl']
                except:
                    _url = self.gis.properties.subscriptionInfo.companionOrganizations[0]['organizationUrl']
                return "https://"+_url
            except AttributeError: 
                return self.gis.url
        except AttributeError:
            print("Hub does not exist or is inaccessible.")
            raise
        
    @property
    def community_org_url(self):
        """
        Returns the AGOL org id of the Community Organization associated with this Hub.
        """
        try:
            self.gis.properties.portalProperties.hub
            try:
                self.gis.properties.portalProperties.hub.settings.communityOrg
                try:
                    _url = self.gis.properties.publicSubscriptionInfo.companionOrganizations[0]['organizationUrl']
                except:
                    _url = self.gis.properties.subscriptionInfo.companionOrganizations[0]['organizationUrl']
                return "https://"+_url
            except AttributeError: 
                return self.gis.url
        except:
            print("Hub does not exist or is inaccessible.")
            raise
    
    @_lazy_property
    def initiatives(self):
        """
        The resource manager for Hub initiatives. See :class:`~arcgis.apps.hub.InitiativeManager`.
        """
        return InitiativeManager(self)
    
class Initiative(collections.OrderedDict):
    """
    Represents an initiative within a Hub. An Initiative supports 
    policy- or activity-oriented goals through workflows, tools and team collaboration.
    """
    
    def __init__(self, gis, initiativeItem):
        """
        Constructs an empty Initiative object
        """
        self.item = initiativeItem
        self._gis = gis
        try:
            self._initiativedict = self.item.get_data()
            pmap = PropertyMap(self._initiativedict)
            self.definition = pmap
        except:
            self.definition = None
            
    def __repr__(self):
        return '<%s title:"%s" owner:%s>' % (type(self).__name__, self.title, self.owner)
       
    @property
    def itemid(self):
        """
        Returns the item id of the initiative item
        """
        return self.item.id
    
    @property
    def title(self):
        """
        Returns the title of the initiative item
        """
        return self.item.title
    
    @property
    def description(self):
        """
        Getter/Setter for the initiative description
        """
        return self.item.description
    
    @description.setter
    def description(self, value):
        self.item.description = value
    
    @property
    def snippet(self):
        """
        Getter/Setter for the initiative snippet
        """
        return self.item.snippet
    
    @snippet.setter
    def snippet(self, value):
        self.item.snippet = value
    
    @property
    def owner(self):
        """
        Returns the owner of the initiative item
        """
        return self.item.owner

    @property
    def tags(self):
        """
        Returns the tags of the initiative item
        """
        return self.item.tags
    
    @property
    def url(self):
        """
        Returns the url of the initiative editor
        """
        return self.item.properties['url']
    
    @property
    def site_url(self):
        """
        Returns the url of the initiative site
        """
        return self.item.url
    
    def followers(self, community_gis=None):
        """
        Fetches the list of followers for initiative. 
        """
        followers = []
        _email = False
        _users_e = self._gis.users.search(query='hubInitiativeId|'+self.itemid, outside_org=True)
        if community_gis is not None:
            _users_c = community_gis.users.search(query='hubInitiativeId|'+self.itemid, outside_org=True)
            _email = True
        for _user in _users_e:
            _temp = {}
            _temp['name'] = _user.fullName
            _temp['username'] = _user.username
            if _email:
                try:
                    _temp['email'] = _user.email
                except AttributeError:
                    for _user_c in _users_c:
                        if _user_c.username==_user.username:
                            try:
                                _temp['email'] = _user_c.email
                            except AttributeError:
                                pass
            followers.append(_temp)
        return followers
            
    
class InitiativeManager(object):
    """
    Helper class for managing initiatives within a Hub. This class is not created by users directly. 
    An instance of this class, called 'initiatives', is available as a property of the Hub object. Users
    call methods on this 'initiatives' object to manipulate (add, get, search, etc) initiatives.
    """
    
    def __init__(self, hub, initiative=None):
        self._hub = hub
        self._gis = self._hub.gis
    
    def get(self, initiative_id):
        """ Returns the initiative object for the specified initiative_id.
        =======================    =============================================================
        **Argument**               **Description**
        -----------------------    -------------------------------------------------------------
        initiative_id              Required string. The initiative itemid.
        =======================    =============================================================
        :return:
            The initiative object if the item is found, None if the item is not found.
        .. code-block:: python
            USAGE EXAMPLE: Fetch an initiative successfully
            initiative1 = myHub.initiatives.get('itemId12345')
            initiative1.item
        """
        initiativeItem =    self._gis.content.get(initiative_id)
        if 'hubInitiative' in initiativeItem.typeKeywords:
            return Initiative(self._gis, initiativeItem)
        else:
            raise TypeError("Item is not a valid initiative or is inaccessible.")
    
    def search(self, scope=None, title=None, owner=None, created=None, modified=None, tags=None):
        """ 
        Searches for initiatives.
        ===============     ====================================================================
        **Argument**        **Description**
        ---------------     --------------------------------------------------------------------
        scope               Optional string. Defines the scope of search.
                            Valid values are 'official', 'community' or 'all'.
        ---------------     --------------------------------------------------------------------
        title               Optional string. Return initiatives with provided string in title.
        ---------------     --------------------------------------------------------------------
        owner               Optional string. Return initiatives owned by a username.
        ---------------     --------------------------------------------------------------------
        created             Optional string. Date the initiative was created.
                            Shown in milliseconds since UNIX epoch.
        ---------------     --------------------------------------------------------------------
        modified            Optional string. Date the initiative was last modified.
                            Shown in milliseconds since UNIX epoch
        ---------------     --------------------------------------------------------------------
        tags                Optional string. User-defined tags that describe the initiative.
        ===============     ====================================================================
        :return:
           A list of matching initiatives.
        """

        initiativelist = []
        
        #Build search query
        query = 'typekeywords:hubInitiative'
        if title!=None:
            query += ' AND title:'+title
        if owner!=None:
            query += ' AND owner:'+owner
        if created!=None:
            query += ' AND created:'+created
        if modified!=None:
            query += ' AND modified:'+modified
        if tags!=None:
            query += ' AND tags:'+tags
        
        #Apply org scope and search
        if scope is None or self._gis.url=='https://www.arcgis.com':
            items = self._gis.content.search(query=query, max_items=5000)
        elif scope.lower()=='official':
            query += ' AND access:public'
            _gis = GIS(self._hub.enterprise_org_url)
            items = _gis.content.search(query=query, max_items=5000)
        elif scope.lower()=='community':
            query += ' AND access:public'
            _gis = GIS(self._hub.community_org_url)
            items = _gis.content.search(query=query, max_items=5000)
        elif scope.lower()=='all':
            items = self._gis.content.search(query=query, outside_org=True, max_items=5000)
        else:
            raise Exception("Invalid value for scope")
            
        #Return searched initiatives
        for item in items:
            initiativelist.append(Initiative(self._gis, item))
        return initiativelist

### 2. Fetch initiatives (topic names)

In [4]:
#Hub setup
myHub = Hub(e_org_url, e_username, e_pasword)

cgis = GIS(c_org_url, c_username, c_password)

In [5]:
all_initiatives = myHub.initiatives.search(scope='official')
len(all_initiatives)

55

### 3. Implement methods making necessary GovDelivery API calls

In [6]:
def get_topic():
    topics = []
    headers = {'Content-Type': 'application/xml'}
    r = requests.get('https://stage-api.govdelivery.com/api/account/MDSHA/topics.xml', headers=headers, auth=(gd_username, gd_password))

    if(r.status_code == 200):
        topics_xml = r.text
        #Parse xml to get topic name
        root = ET.fromstring(topics_xml)
        for topic in root.findall('topic'):
            temp = {}
            temp['topic_name'] = topic.find('name').text
            temp['topic_code'] = topic.find('code').text
            topics.append(temp)
        return topics
    else:
        return None

In [7]:
def create_topic(topic_name, site_url):
    '''Uses govdelivery API to POST topic with topic_name'''
    ###Define XML
    topic_xml = """<topic>
      <code>MDSHA_1</code>
      <name>"""+topic_name+"""</name>
      <short-name>test1</short-name>
      <description nil="true"></description>
      <send-by-email-enabled type="boolean">false</send-by-email-enabled>
      <wireless-enabled type="boolean">false</wireless-enabled>
      <rss-feed-url nil="true"></rss-feed-url> 
      <rss-feed-title nil="true"></rss-feed-title>
      <rss-feed-description nil="true"></rss-feed-description>
      <pagewatch-enabled type="boolean">true</pagewatch-enabled>
      <pagewatch-suspended type="boolean">false</pagewatch-suspended>
      <default-pagewatch-results type="integer" nil="true"></default-pagewatch-results>
      <pagewatch-autosend type="boolean">false</pagewatch-autosend>
      <pagewatch-type type="integer">1</pagewatch-type>
      <watch-tagged-content type="boolean">false</watch-tagged-content>
      <pages type="array">
        <page>
          <url>"""+site_url+"/api/v3/datasets/feed"+"""</url>
        </page>
      </pages>
      <visibility>Listed</visibility>
    </topic>
    """
    
    ###Create topic
    headers = {'Content-Type': 'application/xml'}
    r = requests.post('https://stage-api.govdelivery.com/api/account/MDSHA/topics.xml', data=topic_xml, headers=headers, auth=(gd_username, gd_password))

    if(r.status_code == 200):
        topic_success_xml = r.text
        #Parse xml to get topic name
        xml_output = r.text
        root = ET.fromstring(xml_output)
        return root.find('to-param').text
    elif(r.status_code !=200):
        print("Oops something did not work - here is the error message")
        print(r.status_code)
        print(r.text)
        return None

In [8]:
def update_topic(topic_name, topic_code, site_url):
    '''Uses govdelivery API to POST topic with topic_name'''
    ###Define XML
    topic_xml = """<topic>
      <code>"""+topic_code+"""</code>
      <name>"""+topic_name+"""</name>
      <short-name>test1</short-name>
      <description nil="true"></description>
      <send-by-email-enabled type="boolean">false</send-by-email-enabled>
      <wireless-enabled type="boolean">false</wireless-enabled>
      <rss-feed-url nil="true"></rss-feed-url> 
      <rss-feed-title nil="true"></rss-feed-title>
      <rss-feed-description nil="true"></rss-feed-description>
      <pagewatch-enabled type="boolean">true</pagewatch-enabled>
      <pagewatch-suspended type="boolean">false</pagewatch-suspended>
      <default-pagewatch-results type="integer" nil="true"></default-pagewatch-results>
      <pagewatch-autosend type="boolean">false</pagewatch-autosend>
      <pagewatch-type type="integer">1</pagewatch-type>
      <watch-tagged-content type="boolean">false</watch-tagged-content>
      <pages type="array">
        <page>
          <url>"""+site_url+"/api/v3/datasets/feed"+"""</url>
        </page>
      </pages>
      <visibility>Listed</visibility>
    </topic>
    """
    
    ###Create topic
    headers = {'Content-Type': 'application/xml'}
    r = requests.put('https://stage-api.govdelivery.com/api/account/MDSHA/topics/'+topic_code+'.xml', data=topic_xml, headers=headers, auth=(gd_username, gd_password))
    if(r.status_code !=200):
        print("Oops something did not work - here is the error message")
        print(r.status_code)
        print(r.text)
        return None

In [9]:
def add_subscription(topic_code, email):
    '''Register subscribers for a particular topic'''
    subscriber_xml = """<subscriber>
      <email>"""+email+"""</email>
      <send-notifications type='boolean'>false</send-notifications>
      <topics type='array'>
        <topic>
          <code>"""+topic_code+"""</code>
        </topic>
      </topics>
    </subscriber>
    """
    headers = {'Content-Type': 'application/xml'}
    r = requests.post('https://stage-api.govdelivery.com/api/account/MDSHA/subscriptions.xml', data=subscriber_xml, headers=headers, auth=(gd_username, gd_password))

    if(r.status_code != 200):
        print("Oops something did not work - here is the error message")
        print(r.status_code)
        print(r.text)

In [10]:
def delete_subscription(topic_code, email):
    '''Register subscribers for a particular topic'''
    subscriber_xml = """<subscriber>
      <email>"""+email+"""</email>
      <send-notifications type='boolean'>false</send-notifications>
      <topics type='array'>
        <topic>
          <code>"""+topic_code+"""</code>
        </topic>
      </topics>
    </subscriber>
    """
    headers = {'Content-Type': 'application/xml'}
    r = requests.delete('https://stage-api.govdelivery.com/api/account/MDSHA/subscriptions.xml', data=subscriber_xml, headers=headers, auth=(gd_username, gd_password))

    if(r.status_code != 200):
        print("Oops something did not work - here is the error message")
        print(r.status_code)
        print(r.text)

### 4. Main script that fetches new followers and adds them as subscribers if needed

In [11]:
#Fetch existing topics
existing_topics = get_topic()

for initiative in all_initiatives:
    followers = []
    new_subscribers = []
    unfollowing_subscribers = []
    last_followers = []
    present = []
    #Initiative properties
    initiative_id = initiative.itemid
    #Get initiative followers
    followers = initiative.followers(cgis)
    follower_emails = [follower['email'] for follower in followers]
    
    #Construct initiative page watch url
    url = initiative.site_url
    if url[:5]=='https':
        site_url = "https://" + url[8:]
    else:
        site_url = "https://" + url[7:]
    
    #Construct topic name
    initiative_title = initiative.title.replace(" ","")
    initiative_title = initiative_title.replace("/","")
    topic_name = initiative_title + '_' + initiative_id 
    
    #Create or Update initiative_topic as necessary
    present = [topic for topic in existing_topics if topic['topic_name']==topic_name]
    if not present:
        topic_code = create_topic(topic_name, site_url)
        new_subscribers = follower_emails
    else:
        topic_code = present[0]['topic_code']
        now = datetime.datetime.now()
        minutes_ago = now - datetime.timedelta(minutes=minutes)
        since_last_execution = int(minutes_ago.timestamp() * 1000)
        #Update topic if it has updated since last execution
        if initiative.item.modified > since_last_execution:
            update_topic(topic_name, topic_code, site_url)
            
        #Update existing subscribers list if necessary
        if os.path.exists(topic_name+'.txt'):
            last_followers = [line.rstrip('\n') for line in open(topic_name+'.txt')]
            new_subscribers = list(set(follower_emails) - set(last_followers))
            unfollowing_subscribers = list(set(last_followers) - set(follower_emails))
        
    #Update internal files with current initiaitve followers
    if follower_emails != last_followers:
        with open(topic_name+'.txt', "w+") as f:
            for follower in follower_emails:
                f.write("%s\n" % follower)  
    
    #Add new followers as topic subscribers
    if new_subscribers is not None:
        for email in new_subscribers:
            add_subscription(topic_code, email)
                      
    #Remove subscribers that do not follow anymore
    if unfollowing_subscribers is not None:
        for email in unfollowing_subscribers:
            delete_subscription(topic_code, email)